# Uraniborg

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm

In [ ]:
%reload_kedro
df_stat = catalog.load('uraniborg.trip_statistics_clean')
#df_stat = catalog.load('uraniborg.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True);
#df_stat.describe()

In [ ]:
#df_stat.info()

In [ ]:
df_stat["consumption"] = df_stat["Consumption ME1 (L)"] + df_stat["Consumption ME2 (L)"]

In [ ]:
df_stat['item'] = 'raw'
df_clean = df_stat.copy()
df_clean['item'] = 'clean'
df_clean.dropna(subset=['consumption'], inplace=True)
#mask = df_clean['E'] > df_clean['E'].quantile(0.01) 
#df_clean = df_clean.loc[mask].copy()

In [ ]:
facegrid = sns.relplot(data=df_clean, x='start_time',y='consumption', height=3, aspect=3);
facegrid.set_ylabels('(Fuel consumption per trip) [l]');

In [ ]:
sns.displot(df_clean, x='consumption', hue='trip_direction', aspect=3);

In [ ]:
df_ = df_clean.set_index('start_time')
df_.sort_index(inplace=True)

tänd = '2021-04-01'
utbildning = '2021-05-11'

df_.loc[:tänd,'period'] = 'släckta skärmar'
df_.loc[tänd:utbildning,'period'] = 'tända skärmar'
df_.loc[utbildning:,'period'] = 'efter utbildning'

In [ ]:
df_expanding = df_.expanding().mean()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,7)

#df_before.plot(y='consumption', label='before', ax=ax, style='b.')
#df_after.plot(y='consumption', label='after', ax=ax, style='g.')

styles = ['k.','y.','b.']
for style, (period, df_period) in zip(styles,df_.groupby(by='period', sort=False)):
    df_period.plot(y='consumption', label=period, ax=ax, style=style)



#df_rolling.plot(y='consumption', label=f'rolling mean ({window})', ax=ax, style='m-')
df_expanding.loc[tänd:].plot(y='consumption', label=f'expanding mean', ax=ax, style='k-', lw=3)
ax.set_ylabel('Fuel consumption [liters]')
ax.legend();

In [ ]:
#import plotly.graph_objects as go
#
#fig1 = px.scatter(df_, y='consumption', color='period')
#fig2 = px.line(df_expanding.loc[tänd:], y='consumption')
#fig = go.Figure(fig1.data + fig2.data)
#fig

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,7)

styles = ['k.','y.','b.']
for style, (period, df_period) in zip(styles,df_.groupby(by='period', sort=False)):
    df_period.plot(y='w', label=period, ax=ax, style=style)

df_expanding.loc[tänd:].plot(y='w', label=f'expanding mean', ax=ax, style='k-', lw=3)
ylims = ax.get_ylim()
ax.set_ylim((0,ylims[1]))
ax.set_ylabel('Wind speed [m/s]')
ax.legend();

In [ ]:
interesting = ['sog','consumption','E','distance','w']
df_compare = df_.groupby(by='period', sort=False).mean()[interesting]
df_compare

In [ ]:
M = np.tile(df_compare['consumption'],(len(df_compare),1))
changes = np.triu(M - M.T)
df_changes = pd.DataFrame(changes, index=df_compare.index, columns=df_compare.index)

columns = df_changes.columns[~(df_changes==0).all(axis=0)]
rows = df_changes.index[~(df_changes==0).all(axis=1)]
df_changes = df_changes.loc[rows,columns].copy()

(df_changes/df_compare.loc['släckta skärmar']['consumption'])*100

## Hypothesis testing

In [ ]:
from statsmodels.stats import weightstats as stests

In [ ]:
key = 'consumption'
for direction, df_direction in df_.groupby(by='trip_direction', sort=False):
    
    periods = df_direction.groupby(by='period', sort=False)
    
    z1 = periods.get_group('släckta skärmar')[key]
    z2 = periods.get_group('efter utbildning')[key]
    bins = np.linspace(df_[key].min(),df_[key].max(),100)
    
    fig,ax=plt.subplots()
    grid = sns.histplot(z1, stat="density", bins=bins, label='släckta skärmar', ax=ax);
    grid = sns.histplot(z2, stat="density", bins=bins, color='r', label='efter utbildning', ax=ax)
    fig.suptitle(direction)
    ax.legend();
    
    

In [ ]:
periods = df_.groupby(by='period', sort=False)
    
z1 = periods.get_group('släckta skärmar')[key]
z2 = periods.get_group('efter utbildning')[key]

In [ ]:
z1.mean()

In [ ]:
z2.mean()

In [ ]:
diff = 0.05*z1.mean()
ztest ,pvalue = stests.ztest(z1, z2, value=diff)

In [ ]:
pvalue

In [ ]:
ttest,pvalue,_ = stests.ttest_ind(z1, z2, value=diff)

In [ ]:
pvalue

In [ ]:
N = 20000
z1_means = np.zeros(N)
for i in range(N):
    z1_means[i] = z1.sample(replace=True, n=3).mean()

In [ ]:
z2_means = np.zeros(N)
for i in range(N):
    z2_means[i] = z2.sample(replace=True, n=3).mean()

In [ ]:
fig,ax=plt.subplots()
bins = np.linspace(np.min([np.quantile(z1_means,0.001), np.quantile(z2_means, 0.001)]), np.max([np.quantile(z1_means,0.999),np.quantile(z2_means,0.999)]),100)
sns.histplot(z1_means, kde=True, stat="density", bins=bins, label='släckta skärmar', ax=ax);
sns.histplot(z2_means, kde=True, stat="density", bins=bins, label='efter utbildning', color='r', ax=ax);
ax.set_xlabel('Energy consumption [kJ]')
ax.legend();

In [ ]:
rv1 = norm(loc=z1_means.mean(), scale=z1_means.std())
rv2 = norm(loc=z2_means.mean(), scale=z2_means.std())